In [36]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
import pandas as pd
import re
from transformers import BertTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from langdetect import detect


In [23]:
def clean_text(text):
    # Menghilangkan karakter non-alfanumerik dan mengganti beberapa karakter
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Opsi tambahan: mengganti newline dengan spasi, dll.
    text = text.replace('\n', ' ').replace('\r', '')
    return text

In [24]:
# Inisialisasi Tokenizer mBERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [29]:
def sliding_window_tokenize(text, max_length=700, stride=50):
    # Konversi teks ke token dan inisialisasi segmen
    initial_tokens = tokenizer.encode(text, add_special_tokens=False)
    total_tokens = len(initial_tokens)
    window_segments = []

    if total_tokens <= max_length:
        # Jika total token kurang dari max_length, tidak perlu sliding window
        return [tokenizer.encode(text, add_special_tokens=True)]

    for i in range(0, total_tokens, stride):
        # Memilih segmen token dengan mempertimbangkan token spesial
        window_segment = [tokenizer.cls_token_id] + initial_tokens[i:i + max_length - 2] + [tokenizer.sep_token_id]
        window_segments.append(window_segment)

    return window_segments

In [26]:
# Fungsi untuk menggabungkan token menjadi string
def tokens_to_string(tokens):
    return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokens))

In [27]:
# Memuat dataset
df = pd.read_csv('Merged_data.csv')

In [30]:
# Pembersihan teks
df['cleaned_OCR_text'] = df['OCR_Text'].apply(clean_text)
# Sliding window tokenization
df['tokenized_segments'] = df['cleaned_OCR_text'].apply(sliding_window_tokenize)

In [31]:
# Menggabungkan segmen teks yang telah ditokenisasi
df['processed_text'] = df['tokenized_segments'].apply(lambda segs: ' '.join([tokens_to_string(seg) for seg in segs]))

In [32]:
# Fungsi untuk menghilangkan stop words
def remove_stopwords(text, lang):
    words = text.split()
    if lang == 'id':
        stop_words = set(stopwords.words('indonesian'))
    else:
        stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [39]:
def stem_text(text, lang):
    words = text.split()
    if lang == 'id':
        # Menggunakan Sastrawi untuk bahasa Indonesia
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
    else:
        # Menggunakan SnowballStemmer atau PorterStemmer untuk bahasa lain
        stemmer = SnowballStemmer(lang) if lang in SnowballStemmer.languages else PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

In [40]:
df['language'] = df['processed_text'].apply(lambda x: detect(x) if x.strip() != "" else "")
df['no_stopwords'] = df.apply(lambda x: remove_stopwords(x['processed_text'], x['language']), axis=1)
df['stemmed_text'] = df.apply(lambda x: stem_text(x['no_stopwords'], x['language']), axis=1)